In [1]:
%matplotlib inline
from figures.demos_basic import single_trial

single_trial(
    SEED=0,
    N_ITEM_UNITS=100,
    N_MEMORY_UNITS=2000,
    P_CXN=0.2,
    PAIRS=[(0, 1), (2, 3)])

ImportError: No module named figures.demos_basic